# 1. 아래의 데이터에 대해 Merkle Root 값을 구하시오

* txA = 'Hello'

* txB = 'How are you?'

### 1.1 txA, txB를 재해싱(더블 해싱)하세요 (_hA, _hB)

In [11]:
import hashlib
txA = 'Hello'
txB = 'How are you?'

_hA = hashlib.sha256(hashlib.sha256(txA.encode()).digest()).hexdigest() #재해싱된 값
_hB = hashlib.sha256(hashlib.sha256(txB.encode()).digest()).hexdigest()

### 1.2 _hA, _hB를 바이트교환 하세요 (hA, hB)

In [13]:
print("바이트 교환 전 A = " + _hA) #바이트 교환 전
print("바이트 교환 전 B = " + _hB)

hA = "".join(reversed([_hA[i:i+2] for i in range(0, hashlib.sha256(txA.encode()).digest_size*2, 2)]))
hB = "".join(reversed([_hB[i:i+2] for i in range(0, hashlib.sha256(txB.encode()).digest_size*2, 2)]))

print("바이트 교환 후 A = " + hA) #바이트 교환 후
print("바이트 교환 후 B = " + hB)

바이트 교환 전 A = 70bc18bef5ae66b72d1995f8db90a583a60d77b4066e4653f1cead613025861c
바이트 교환 전 B = 1027ce357faf49fc632d35908db181ae951aa3df9060595d94fb33769d71e7d0
바이트 교환 후 A = 1c86253061adcef153466e06b4770da683a590dbf895192db766aef5be18bc70
바이트 교환 후 B = d0e7719d7633fb945d596090dfa31a95ae81b18d90352d63fc49af7f35ce2710


### 1.3 hA + hB를 하고 (_hAhB) 재해싱하세요 (_hAB).

In [14]:
_hAhB = hA + hB

_hAB = hashlib.sha256(hashlib.sha256(_hAhB.encode()).digest()).hexdigest()

### 1.4 _hAB를 바이트교환 하세요.(hAB)

In [15]:
hAB = "".join(reversed([_hAB[i:i+2] for i in range(0, hashlib.sha256(_hAhB.encode()).digest_size*2, 2)]))
print("Merkle Root 값은 = " + hAB)

Merkle Root 값은 = 032916ba0fc9c63fc5440a3b20069370da1b5d3c605797b28ac1a5a5876dc7e0


# 2. 친구에게  주소를 구하고, 그 주소로 송금해 보자. 송금이 되지 않으면 왜 안되는지 이유를 알아보자.

In [16]:
!geth --exec "eth.accounts" attach http://localhost:8445

["0x60c0ca8790e0177b4aedbd37a4079bca28e41375", "0x2c647c0376a1123d89d07a4fa494acd44b3d2975", "0x474c4efb8e4b7fbbf2642029f1825898a2c3fa54", "0x1dad809e9812e5c4bf22445241cb6c434243c952", "0xd66e3df975c2599b98f607e49c48954bf1beb3d8"]


In [55]:
%%writefile src/sameAccount-tran.js
var peer = "0x2c647c0376a1123d89d07a4fa494acd44b3d2975"
var bal1 = eth.getBalance(eth.coinbase);
var bal2 = eth.getBalance(peer);
console.log('my account balance in ether: ', web3.fromWei(bal1, "ether"));
console.log('peer balance in ether: ', web3.fromWei(bal2, "ether"));
console.log('block number: ', eth.blockNumber);
console.log('transaction count: ', eth.getTransactionCount(eth.coinbase));

var h = eth.sendTransaction({from:eth.coinbase, to:peer, value:10000});
console.log("\n...mining start");
miner.start();admin.sleepBlocks(1);miner.stop();
console.log("mining done...\n");
var bal1new = eth.getBalance(eth.coinbase);
var bal2new = eth.getBalance(eth.accounts[1]);
console.log('my account balance in ether: ', web3.fromWei(bal1new, "ether"));
console.log('peer balance in ether: ', web3.fromWei(bal2new, "ether"));
console.log('block number: ', eth.blockNumber);
console.log('transaction count: ', eth.getTransactionCount(eth.coinbase));

console.log("\n\nincreased balance: ", bal2new - bal2);
console.log("gasUsed: ", eth.getTransactionReceipt(h).gasUsed);

Overwriting src/sameAccount-tran.js


In [56]:
!geth --exec "loadScript('src/sameAccount-tran.js')" attach http://localhost:8445

my account balance in ether:  696.4062499999997055
peer balance in ether:  285.000000000000271
block number:  246
transaction count:  20

...mining start
mining done...

my account balance in ether:  714.8437499999996855
peer balance in ether:  285.000000000000291
block number:  248
transaction count:  22


increased balance:  32768
gasUsed:  21000
true


# 3. 마이닝
해시는 100미만의 양수로 정해진다고 하자.

NONCE는 반복회수로만 쓰이고 무작위 수를 생성하는데 입력되지는 않는다.

목표해시는 난이도에 따라 결정이 된다고 하자.

마이닝을 해서, 목표 해시를 찾아보자.

### 3.1 90을 목표해시로 잡고 몇 회만에 마이닝 성공하는지 출력

In [57]:
import hashlib
import random

found = False
NONCE = 0

while found == False:
    guessHash = random.randint(1, 99)
    if (guessHash < 90):
        found = True
    NONCE += 1
    print("NONCE: ",NONCE, "guessHash: ",guessHash)
print("Solved NONCE: ", NONCE, "guessHash: ", guessHash)
print("90을 목표해시로 정했을 경우", NONCE, "회 만에 마이닝에 성공")

NONCE:  1 guessHash:  88
Solved NONCE:  1 guessHash:  88
90을 목표해시로 정했을 경우 1 회 만에 마이닝에 성공


### 3.2 10을 목표해시로 잡고 몇 회만에 마이닝 성공하는지 출력

In [58]:
found = False
NONCE = 0

while found == False:
    guessHash = random.randint(1, 99)
    if (guessHash < 10):
        found = True
    NONCE += 1
    print("NONCE: ",NONCE, "guessHash: ",guessHash)
print("Solved NONCE: ", NONCE, "guessHash: ", guessHash)
print("10을 목표해시로 정했을 경우", NONCE, "회 만에 마이닝에 성공")

NONCE:  1 guessHash:  48
NONCE:  2 guessHash:  40
NONCE:  3 guessHash:  44
NONCE:  4 guessHash:  28
NONCE:  5 guessHash:  6
Solved NONCE:  5 guessHash:  6
10을 목표해시로 정했을 경우 5 회 만에 마이닝에 성공


### 3.3 찾고자 하는 해시가 0000으로 시작하면 몇 회만에 찾는지 출력

In [59]:
import hashlib
ntry=1   #시도 횟수
found=False  #bollen type
blockNumber=54 # hex 임의
NONCE=0
data='Hello'
previousHash='5d7c7ba21cbbcd75d14800b100252d5b428e5b1213d27c385bc141ca6b47989e'
#이전 블록 헤더의 해쉬값
while found==False: #발견할 동안 Ture가 될 동안 반복
    z=str(blockNumber)+str(NONCE)+data+previousHash #1) 블록헤더
    guessHash=hashlib.sha256(z.encode('utf-8')).hexdigest()#2)재해싱
    if guessHash[:4]=='0000':#3) Target Hash와 비교
        found=True
    NONCE+=1 #안되면 난스값 하나 추가해서 다시 한번더
    if(NONCE%10000000==0):   #print guessHash every 10000000
        print("NONCE: ",NONCE, guessHash) #이 시기마다 한 번씩 추출력해서 확인
print("Solved ", "NONCE: ", NONCE, "guessHash: ", guessHash)

Solved  NONCE:  94280 guessHash:  000043ce4a61d02bff0e68ba18a7daf448cb3b93691fdd4850f6cd3f85b7a13f


### 3.4 찾고자 하는 해시가 00000으로 시작하면 몇 회만에 찾는지 출력

In [60]:
import hashlib
ntry=1
found=False
blockNumber=54 # hex
NONCE=0
data='Hello'
previousHash='5d7c7ba21cbbcd75d14800b100252d5b428e5b1213d27c385bc141ca6b47989e'
while found==False:
    z=str(blockNumber)+str(NONCE)+data+previousHash
    guessHash=hashlib.sha256(z.encode('utf-8')).hexdigest()
    if guessHash[:5]=='00000':
        found=True
    NONCE+=1
    if(NONCE%10000000==0):   #print guessHash every 10000000
        print("NONCE: ",NONCE, guessHash)
print("Solved ", "NONCE: ", NONCE, "guessHash: ", guessHash)

Solved  NONCE:  315753 guessHash:  000007f9f69a43f1bb6ab92672d873b93d6bafaa2007e44b6151bd19efadf4d1


### 3.5 찾고자 하는 해시가 000000으로 시작하면 몇 회만에 찾는지 출력

In [61]:
import hashlib
ntry=1
found=False
blockNumber=54 # hex
NONCE=0
data='Hello'
previousHash='5d7c7ba21cbbcd75d14800b100252d5b428e5b1213d27c385bc141ca6b47989e'
while found==False:
    z=str(blockNumber)+str(NONCE)+data+previousHash
    guessHash=hashlib.sha256(z.encode('utf-8')).hexdigest()
    if guessHash[:6]=='000000':
        found=True
    NONCE+=1
    if(NONCE%10000000==0):   #print guessHash every 10000000
        print("NONCE: ",NONCE, guessHash)
print("Solved ", "NONCE: ", NONCE, "guessHash: ", guessHash)

NONCE:  10000000 660b9e057377381579f5c54347901cf462fce656c069a4d0f26bdd4cf1e05e66
NONCE:  20000000 c31d3ddd7bb92312bcc2a88263d92e94c6d17a641ac9e59ac262c775c7f58925
NONCE:  30000000 236077716f4ce428ec83f12bf74fb7ab76450bad60bc3f21d924e200c39a9fa4
NONCE:  40000000 5b07929bd7088f11d6caf0e65a6f44ab3b7f82d32900430cc7c7244b2917e04d
Solved  NONCE:  45576417 guessHash:  0000003d02b95604bb1ec436ff20e08168dd339f2ec0f9941bfc58bad039994e
